<a href="https://colab.research.google.com/github/liveness-detection-bangkit-team/liveness_ai/blob/main/spoof_modelmaker_ariel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Python 3.10.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: Operation cancelled by user


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from keras.preprocessing.image import load_img, img_to_array
from google.colab.patches import cv2_imshow
import seaborn as sns
import os
from mediapipe_model_maker import image_classifier
#from ai_edge_litert.interpreter import Interpreter
#from tflite_support.metadata_writers import image_classifier
#from tflite_support.metadata_writers import writer_utils
print(tf.__version__)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


2.15.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path ke direktori dataset
train_dir = '/content/drive/MyDrive/datasets/datasets/train_spoof'
test_dir = '/content/drive/MyDrive/datasets/datasets/test_spoof'
MODEL_PATH = "/content/drive/MyDrive/Model_ariel"

In [ ]:
print(train_dir)
labels = []
for i in os.listdir(train_dir):
  if os.path.isdir(os.path.join(train_dir, i)):
    labels.append(i)
print(labels)

/content/drive/MyDrive/datasets/datasets/train_spoof
['real', 'fake']


In [ ]:
train_data = image_classifier.Dataset.from_folder(train_dir)
train_data, validation_data = train_data.split(0.8)
test_data = image_classifier.Dataset.from_folder(test_dir)

In [ ]:
len(train_data), len(validation_data), len(test_data)

(4766, 1192, 100)

In [ ]:
spec = image_classifier.SupportedModels.MOBILENET_V2

In [ ]:
hparams=image_classifier.HParams(learning_rate = 0.001,
                                 epochs=5,
                                 export_dir=os.path.join(MODEL_PATH, 'spoof_model_maker_ariel1'),
                                 batch_size=32,
                                 do_fine_tuning=True,
                                 shuffle=True,
                                 do_data_augmentation=False)
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)
options.model_options = image_classifier.ModelOptions(dropout_rate = 0.2)
model_2 = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2226434 (8.49 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________
None
Epoch 1/5
148/148 [==============================] - 2324s 15s/step - loss: 0.5765 - accuracy: 0.8047 - val_loss: 0.4322 - val_accuracy: 0.9287
Epoch 2/5
148/148 [==============================] - 350s 2s/step - loss: 0.3504 - accuracy: 0.9766 - val_loss: 0.3409 - val_accu

In [ ]:
loss, acc = model_2.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

4/4 [==============================] - 38s 9s/step - loss: 0.3453 - accuracy: 0.9700
Test loss:0.3452554941177368, Test accuracy:0.9700000286102295


In [ ]:
from mediapipe_model_maker import quantization

In [ ]:
quantization_config = quantization.QuantizationConfig.for_float16()

In [ ]:
model_2.export_model(model_name="spoof_mobilenetv2_fp16_planB_ariel1.tflite", quantization_config=quantization_config)